In [1]:
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
import pandas as pd
import re
from multiprocessing import Pool
import pickle

## Define elements for scraping

In [3]:
seasons = [str(season) for season in range(2003, 2022)]

my_headers = {
    'authority': 'cleaningtheglass.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
    'cookie': 'wordpress_logged_in_cfb45adfd4102bd74a046b78d76db012=zach225793%7C1670421579%7Cqe3pn3baHEn8lPIIQEpw2tAfbqsdUfINheMSscIPNl5%7Cce0105c6a9be760e3c27785cfabe16ec680f440f05609b1e800ffd1f561775b0; sessionid=yh4dn9hkv9ohcjzjge5twtl0g5skpi2m; csrftoken=DzYXKBmEics8LjNpMZl7B7szJOBuxQOMtO496G8DI2aSwt9pdwJ4TvUkrdAY8GMj',
}

In [4]:
url = 'https://cleaningtheglass.com/stats/players/?stat_category=onoff_opponent_halfcourt_putbacks&seasontype=regseason&season=2019'

In [5]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

response = session.get(url, headers=my_headers)
html_soup = BeautifulSoup(response.text, 'html.parser')
options = html_soup.find('select').find_all('option')

In [6]:
tabs = [re.search(r'stat_category=\w+', str(option))[0][14:] for option in options]

In [12]:
html_soup.find('stat_table_container')

In [19]:
html_soup.find_all('td')

[<td class="intro_stat player_name" v-cloak=""><a :href="`/stats/player/${player.player_id}`">{{ player.first_name + ' ' + player.last_name }}</a></td>,
 <td class="intro_stat" v-cloak="">{{ player.age_in_season | ageFormat(1) }}</td>,
 <td class="intro_stat" v-cloak=""><a :href="`/stats/team/${player.team_id}`">{{ player.abbr }}</a></td>,
 <td class="intro_stat" v-cloak="">{{ player.pos_category | capitalize }}</td>,
 <td class="intro_stat stat" v-cloak="">{{ player.minutes | floorMinutes }}</td>,
 <td class="stat">{{ statCol | printOnOffStat(player) }}</td>,
 <td :style="$options.filters.percentileColorFromDictionary(statCol, player)" class="stat rank">
                                                 {{ statCol | percentileFromDictionary(player)}}
                                             </td>,
 <td class="stat value">{{ statCol | printOnOffStat(player) }}</td>,
 <td class="spacer"></td>,
 <td class="intro_stat player_name" v-cloak=""><a :href="`/stats/player/${player.player_id}

In [15]:
pd.read_html(str(html_soup.find(id='players-table')[0]))

KeyError: 0

In [24]:
def data_per_tab(season):
    season_tables = []
    for tab in tabs:

        # start session and create response
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        url = "https://cleaningtheglass.com/stats/players/?stat_category=" + tab + "&seasontype=regseason&season=" + season
        response = session.get(url, headers=my_headers)
        response_status = re.search(r'\d+', str(response))[0]
        if response_status != '200': # handle fail case from response
            break

        # parse
        html_soup = BeautifulSoup(response.text, 'html.parser')
        table = pd.read_html(str(html_soup.find_all('table')[0]))[0]
        ids = [re.search(r'\d+', str(html_soup.find('table').find_all('a')[i]))[0] for i in range(len(table))]
        table['id'] = ids
        table.columns = [' '.join(col).strip() for col in table.columns.values]

        session.keep_alive = False

        season_tables.append(table)
        print('success')
    return season_tables

with Pool(12) as p:
    stat_tables = p.map(data_per_tab, seasons)

TypeError: 'NoneType' object is not subscriptable

In [89]:
with open('player_stats.pickle', 'wb') as handle:
    pickle.dump(stat_tables, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import pandas as pd
import re
from multiprocessing import Pool
import pickle

## Define elements for scraping

tabs = ['offensive_overview',
        'shooting_overall',
        'shooting_frequency',
        'shooting_accuracy',
        'foul_drawing']
seasons = [str(season) for season in range(2003, 2022)]

my_headers = {
    'authority': 'cleaningtheglass.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
    'cookie': 'wordpress_logged_in_cfb45adfd4102bd74a046b78d76db012=zach225793%7C1670421579%7Cqe3pn3baHEn8lPIIQEpw2tAfbqsdUfINheMSscIPNl5%7Cce0105c6a9be760e3c27785cfabe16ec680f440f05609b1e800ffd1f561775b0; sessionid=yh4dn9hkv9ohcjzjge5twtl0g5skpi2m; csrftoken=DzYXKBmEics8LjNpMZl7B7szJOBuxQOMtO496G8DI2aSwt9pdwJ4TvUkrdAY8GMj',
}

def data_per_team(season):
    season_tables = []
    for team in range(1, 31):
        team_tables = []
        for tab in tabs:
            
            # start session and create response
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            session.mount('https://', adapter)
            
            url = "https://cleaningtheglass.com/stats/team/" + str(team) + "?season=" + season + "#tab-" + tab
            response = session.get(url, headers=my_headers)
            response_status = re.search(r'\d+', str(response))[0]
            if response_status != '200': # handle fail case from response
                break

            # parse
            html_soup = BeautifulSoup(response.text, 'html.parser')
            table = pd.read_html(str(html_soup.find_all('table')[0]))[0]
            ids = [re.search(r'\d+', str(html_soup.find('table').find_all('a')[i]))[0] for i in range(len(table))]
            table['id'] = ids
            table.columns = [' '.join(col).strip() for col in table.columns.values]
            team_tables.append(table)
            
            session.keep_alive = False
        season_tables.append(team_tables)
    return season_tables

with Pool(12) as p:
    stat_tables = p.map(data_per_team, seasons)

with open('player_stats.pickle', 'wb') as handle:
    pickle.dump(stat_tables, handle, protocol=pickle.HIGHEST_PROTOCOL)ingtheglass.com/stats/players/?stat_category=offensive_overview&seasontype=regseason&season=2019#